In [17]:
import pandas as pd
import numpy as np
import math
import json
from collections import defaultdict
from collections import Counter

In [27]:
df = pd.read_csv("~/Data/anime.csv")
print(df.head())
data, item_to_genre, genres = df[["anime_id", "genre"]].fillna(""), {}, []
for st in data.values:
    item_to_genre[st[0]] = [a.strip() for a in st[1].split(",") if len(a) > 0]
    genres += st[1].split(",")

num_genres = len(set(genres))

   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


In [54]:
rating_df = pd.read_csv("~/Data/clean_rating4.csv").drop(["id"], axis = 1)

In [100]:
users = list(set(user_item.user_id.tolist()))
results, inputs = {}, {}
for epoch in range(10):
    print(epoch + 1)
    test_user = np.random.choice(a = users, size = 1)
    user_item = rating_df[rating_df.user_id != test_user[0]]
    from collections import defaultdict
    grouped, corpus, genre_to_user = user_item.groupby("user_id"), {}, defaultdict(list)
    for gp in grouped.groups:
        items, sentences = [t[1] for t in grouped.get_group(gp).values if t[2] > 5], []
        for item in items:
            sentences += item_to_genre.get(item, [])

        for genre in set(sentences):
            genre_to_user[genre].append(gp)

        corpus[gp] = sentences
        
    sorted_tf_idf, base_genres = tfidf(corpus)
    input = get_input(test_user[0], base_genres, item_to_genre, rating_df, df)
    
    if inputs.get(tuple(input), None):
        result = inputs[tuple(input)]
    else:
        result = recomm(input, corpus, genre_to_user, user_item, sorted_tf_idf)
    
    results[user] = result
    inputs[tuple(input)] = result

    
print(results)

1
['yes', 'yes', 'yes', 'yes', 'no']
2
['no', 'no', 'no', 'no', 'no']
3
4
['no', 'yes', 'no', 'no', 'no']
5
6
7
8
9
10
[[199, 853, 1535, 6746, 431, 1575, 2904, 4898, 5114, 16498], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [5114, 9253, 6547, 1575, 4181, 2904, 4224, 1535, 16498, 19815], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114], [9253, 6547, 1575, 19815, 2904, 4224, 11757, 1535, 4181, 5114]]


In [83]:
def tfidf(corpus):
    word_frequency, doc_frequency, word_tf, word_idf = defaultdict(int), defaultdict(int), defaultdict(float), defaultdict(float)
    for st in corpus.values():
        for word in st:
            word_frequency[word] += 1
        for word in set(st):
            doc_frequency[word] += 1

    num_words = sum(word_frequency.values())
    for word in word_frequency:
        word_tf[word] = word_frequency[word] / num_words

    for doc in doc_frequency:
        word_idf[doc] = math.log(len(corpus) / (doc_frequency[doc] + 1))

    tf_idf = {}
    for word in word_tf:
        tf_idf[word] = word_tf[word] * word_idf[word]
    
    sorted_tf_idf = sorted(tf_idf.items(), key = lambda x: x[1], reverse = True)
    return sorted_tf_idf, [g[0] for g in sorted(tf_idf.items(), key = lambda x: x[1], reverse = True)[:5]]
# dic = {'genre': [i[0] for i in sorted_tf_idf], 'tf_idf': [i[1] for i in sorted_tf_idf]}
# df = pd.DataFrame(dic)
# df.to_csv("tf_idf.csv")

In [63]:
def get_input(user, base_genres, item_to_genre, rating_df, anime_df):
    items = [data[1] for data in rating_df[rating_df.user_id == user].values if data[2] == 10]
#     print(user)
    genres = sum([item_to_genre[i] for i in items], [])
#     liked_genres = [g[0] for g in sorted(Counter(genres).items(), key = lambda x: x[1], reverse = True)[:5]]
#     print(liked_genres)
    result = []
    for g in base_genres:
        if g in genres:
            result.append('yes')
        else:
            result.append('no')
    
    return result

In [98]:
def recomm(input, corpus, genre_to_user, user_item, sorted_tf_idf): 
    print(input)
    candidate, all_user = set(corpus.keys()), set(corpus.keys())
#     print(candidate)
    liked_genres = []
    for idx, ans in enumerate(input):
        if ans == "yes":
            candidate &= set(genre_to_user[sorted_tf_idf[idx][0]])
            liked_genres.append(sorted_tf_idf[idx][0])
        else:
            candidate &= all_user - set(genre_to_user[sorted_tf_idf[idx][0]])
            
#     print(len(candidate))
    rec= defaultdict(int)
    for user in candidate:
        animes = user_item[user_item.user_id == user].anime_id.tolist()
        for item in animes:
            overlap = set(item_to_genre[item]) & set(liked_genres)
            if overlap:
                rec[item] += (int(user_item[(user_item.user_id == user) & (user_item.anime_id == item)].rating.values) - 5) * (len(overlap) / len(liked_genres))
            else:
                rec[item] += (int(user_item[(user_item.user_id == user) & (user_item.anime_id == item)].rating.values) - 5)
    return [g[0] for g in sorted(rec.items(), key = lambda x: x[1], reverse = True)[:10]]

In [101]:
def random_rec(df):
    animes = list(set(df.anime_id))
    rec = np.random.choice(animes, 10)
    return rec

In [102]:
def get_hot_items():
    animes = pd.read_csv("~/Data/anime.csv")
    animes = animes[animes["anime_id"].isin(idx_to_animes)].loc[:, ["anime_id", "rating", "members"]].fillna(0)

    scalar = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
    animes["rating_norm"] = animes[["rating"]].apply(scalar)
    animes["members_norm"] = animes[["members"]].apply(scalar)
    animes["weight"] = 0.6 * animes["rating_norm"] + 0.4 * animes["members_norm"]
    animes = animes.sort_values(by = "weight", ascending = False)
    
    return {anime_to_idx[i[0]]: i[1] for i in animes[["anime_id", "weight"]].values}

In [ ]:
def rec_hot():
    